In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [2]:
# Load the dataset
data = pd.read_csv('diabetic_data.csv', na_values=['NA','?','None'])
data

/var/folders/6f/hrr6km6x5l39n_12_r7994t00000gn/T/ipykernel_14100/1736140095.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('diabetic_data.csv', na_values=['NA','?','None'])


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),NaN,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),NaN,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),NaN,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),NaN,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [3]:
# Display of the first few rows of the dataset to understand its structure
print(data.head())

   encounter_id  patient_nbr             race  gender      age weight  \
0       2278392      8222157        Caucasian  Female   [0-10)    NaN   
1        149190     55629189        Caucasian  Female  [10-20)    NaN   
2         64410     86047875  AfricanAmerican  Female  [20-30)    NaN   
3        500364     82442376        Caucasian    Male  [30-40)    NaN   
4         16680     42519267        Caucasian    Male  [40-50)    NaN   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        25                    1   
1                  1                         1                    7   
2                  1                         1                    7   
3                  1                         1                    7   
4                  1                         1                    7   

   time_in_hospital  ... citoglipton insulin  glyburide-metformin  \
0                 1  ...          No      No                   No

In [4]:
#To check the shape of data 
print(data.shape)

(101766, 50)


In [5]:
# Basic statistics of the dataset
print(data.describe())

       encounter_id   patient_nbr  admission_type_id  \
count  1.017660e+05  1.017660e+05      101766.000000   
mean   1.652016e+08  5.433040e+07           2.024006   
std    1.026403e+08  3.869636e+07           1.445403   
min    1.252200e+04  1.350000e+02           1.000000   
25%    8.496119e+07  2.341322e+07           1.000000   
50%    1.523890e+08  4.550514e+07           1.000000   
75%    2.302709e+08  8.754595e+07           3.000000   
max    4.438672e+08  1.895026e+08           8.000000   

       discharge_disposition_id  admission_source_id  time_in_hospital  \
count             101766.000000        101766.000000     101766.000000   
mean                   3.715642             5.754437          4.395987   
std                    5.280166             4.064081          2.985108   
min                    1.000000             1.000000          1.000000   
25%                    1.000000             1.000000          2.000000   
50%                    1.000000             7.00000

In [6]:
#Seeing the summary of the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      99493 non-null   object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    3197 non-null    object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                61510 non-null   object
 11  medical_specialty         51817 non-null   object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

In [7]:
# To remove the column 
columns_to_remove = ['weight', 'max_glu_serum', 'A1Cresult', 'encounter_id', 'patient_nbr']

# Remove specific columns
data_cleaned = data.drop(columns=columns_to_remove)

# Showing the data Shape 
data_cleaned.shape

(101766, 45)

In [13]:
from sklearn.cluster import KMeans
import pandas as pd

# Selecting only the columns with continuous values and no missing values
continuous_columns = data_cleaned.select_dtypes(include=['float64', 'int64']).dropna(axis=1)

# Check if there are any missing values after dropping columns
if continuous_columns.isnull().sum().sum() > 0:
    print("There are still missing values in the continuous columns. Please handle them first.")
else:
    # Chooseing the number of clusters
    num_clusters = 3

    # Initialize the K-Means clustering algorithm
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)

    # Fit the model to the data
    clusters = kmeans.fit_predict(continuous_columns)

    # Print cluster centers
    print("Cluster Centers:")
    print(kmeans.cluster_centers_)

    # Print cluster assignments
    print("Cluster Assignments:")
    print(clusters)

    # adding cluster assignments to the original dataframe
    data_with_clusters = data_cleaned.copy()
    data_with_clusters['Cluster'] = clusters

    # Print the data with cluster labels
    print("Data with Cluster Labels:")
    print(data_with_clusters)


Cluster Centers:
[[ 1.8816502   3.61533215  5.04631888  3.93656732 41.05450374  1.2778306
  14.91341821  0.31672936  0.18579224  0.64098218  7.21117358]
 [ 2.62484497  3.68775115  6.70184055  3.32385772 13.31224885  1.24229796
  13.43572952  0.44341916  0.21545865  0.52869971  7.14987349]
 [ 1.86679508  3.87862281  6.21079243  5.71535197 64.13570289  1.48896981
  19.20468294  0.40105938  0.20468294  0.6925091   7.89592801]]
Cluster Assignments:
[0 2 1 ... 0 0 1]
Data with Cluster Labels:
                   race  gender      age  admission_type_id  \
0             Caucasian  Female   [0-10)                  6   
1             Caucasian  Female  [10-20)                  1   
2       AfricanAmerican  Female  [20-30)                  1   
3             Caucasian    Male  [30-40)                  1   
4             Caucasian    Male  [40-50)                  1   
...                 ...     ...      ...                ...   
101761  AfricanAmerican    Male  [70-80)                  1   
101

In [15]:
# For dealing with missing values 
# Identify columns with missing values
columns_with_missing_values = data_with_clusters.columns[data_with_clusters.isna().any()].tolist()
print(columns_with_missing_values)

['race', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3']


In [16]:
from sklearn.impute import SimpleImputer

# For each column, imputeing missing values based on the cluster
for cluster in data_with_clusters['Cluster'].unique():
    # Selecting the cluster
    cluster_data = data_with_clusters[data_with_clusters['Cluster'] == cluster]
    
    # Iterate over columns with missing values
    for column in columns_with_missing_values:
        # Determine the imputation strategy based on data type
        if cluster_data[column].dtype == "float64" or cluster_data[column].dtype == "int64":
            # For numerical columns impute mean
            strategy = 'mean'  
        else:
            # For categorical columns mode    
            strategy = 'most_frequent'
        
        # Initialize the imputer
        imputer = SimpleImputer(strategy=strategy)
        
        # Perform imputation on the selected column
        imputed_values = imputer.fit_transform(cluster_data[[column]])
        
        # Update the DataFrame with the imputed values
        data_with_clusters.loc[data_with_clusters['Cluster'] == cluster, column] = imputed_values.flatten()

# Verify if there are still missing values
print(data_with_clusters.isnull().sum())


race                        0
gender                      0
age                         0
admission_type_id           0
discharge_disposition_id    0
admission_source_id         0
time_in_hospital            0
payer_code                  0
medical_specialty           0
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
diag_1                      0
diag_2                      0
diag_3                      0
number_diagnoses            0
metformin                   0
repaglinide                 0
nateglinide                 0
chlorpropamide              0
glimepiride                 0
acetohexamide               0
glipizide                   0
glyburide                   0
tolbutamide                 0
pioglitazone                0
rosiglitazone               0
acarbose                    0
miglitol                    0
troglitazone                0
tolazamide

In [30]:
# Separate numerical and categorical columns
numerical_columns = data_with_clusters.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = data_with_clusters.select_dtypes(include=['object']).columns

print("Numerical Columns:")
print(numerical_columns)
print("\nCategorical Columns:")
print(categorical_columns)

Numerical Columns:
Index(['admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses'],
      dtype='object')

Categorical Columns:
Index(['race', 'gender', 'age', 'payer_code', 'medical_specialty', 'diag_1',
       'diag_2', 'diag_3', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')


In [33]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes
import numpy as np

#  Preprocess the Data
# Standardize Numerical Columns
scaler = StandardScaler()
numerical_data_standardized = scaler.fit_transform(data_with_clusters[numerical_columns])

# One-hot encode Categorical Columns
categorical_data_encoded = pd.get_dummies(data_with_clusters[categorical_columns])

# Merge Numerical and Categorical Data
merged_data = np.concatenate((numerical_data_standardized, categorical_data_encoded), axis=1)

# Perform K-Prototype Clustering
# Initialize KPrototypes
kproto = KPrototypes(n_clusters=3, init='Cao')

# Fit the model
clusters = kproto.fit_predict(merged_data, categorical=list(range(len(categorical_columns))))

# Print cluster centroids
print("Cluster Centroids:")
print(kproto.cluster_centroids_)

# Print cluster assignments for each data point
print("Cluster Assignments:")
print(clusters)

# Add cluster labels to the original DataFrame
data_with_clusters['Cluster_Labels'] = clusters

# Print the DataFrame with cluster labels
print("Data with Cluster Labels:")
print(data_with_clusters)



Cluster Centroids:
[[2.12716173e-05 5.33492161e-02 6.79053838e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 6.40150549e-02 7.45279408e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.51613041e-02 7.43750801e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
Cluster Assignments:
[2 0 1 ... 0 0 2]
Data with Cluster Labels:
                   race  gender      age  admission_type_id  \
0             Caucasian  Female   [0-10)                  6   
1             Caucasian  Female  [10-20)                  1   
2       AfricanAmerican  Female  [20-30)                  1   
3             Caucasian    Male  [30-40)                  1   
4             Caucasian    Male  [40-50)                  1   
...                 ...     ...      ...                ...   
101761  AfricanAmerican    Male  [70-80)                  1   
101762  AfricanAmerican  Female  [80-90)                  1   
101763        Caucasian    Male  [70-80)           

In [35]:
import os

# Specify the file name along with the directory path
file_path = '/Users/luffy_015/Desktop/data_with_clusters.csv'

# Checking if the directory exists, if not, create it
directory = os.path.dirname(file_path)
if not os.path.exists(directory):
    os.makedirs(directory)

# To Save the DataFrame to a CSV file
data_with_clusters.to_csv(file_path, index=False)

print(f'Data saved to {file_path}')

Data saved to /Users/luffy_015/Desktop/data_with_clusters.csv
